In [1]:
from datasets import load_dataset
from underthesea import sent_tokenize
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm import tqdm

In [ ]:
from datasets import load_dataset

raw_dataset = load_dataset("vietlegalqa/tvpl_summary_1k_231021")

In [6]:
import json

json.dump(
    raw_dataset["train"].to_list(),
    open(
        "./data/tvpl_summary_1k.json",
        "w",
        encoding="utf-8",
    ),
    indent=4,
    ensure_ascii=False,
)

In [3]:
dataset = (
    raw_dataset["train"]
    .rename_column("context_title_question", "context")
    .rename_column("answers", "document")
    .remove_columns(["questions", "documents", "title_question"])
)
dataset

Dataset({
    features: ['url', 'title', 'context', 'document'],
    num_rows: 151879
})

In [4]:
tokenizer = AutoTokenizer.from_pretrained("VietAI/vit5-large-vietnews-summarization")
model = AutoModelForSeq2SeqLM.from_pretrained(
    "VietAI/vit5-large-vietnews-summarization"
)
model.cuda()

T5ForConditionalGeneration(
  (shared): Embedding(36096, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(36096, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=1024, out_features=2816, bias=False)
              (wi_1): Linear(in_features=1024, out_features=2816, bias=False)
       

In [5]:
def summary_sents(sents):
    sents = f"vietnews: {sents} </s>"

    encoding = tokenizer(
        sents,
        return_tensors="pt",
    )

    input_ids, attention_masks = encoding["input_ids"].to("cuda"), encoding[
        "attention_mask"
    ].to("cuda")

    outputs = model.generate(
        input_ids=input_ids,
        attention_mask=attention_masks,
        max_length=len(sents),
    )
    result = []
    for output in outputs:
        result.append(
            tokenizer.decode(
                output, skip_special_tokens=True, clean_up_tokenization_spaces=True
            )
        )
    return result

In [6]:
def summary_example(example):
    summaries = example["summary"] if isinstance(example["summary"], list) else []
    sents = str()
    sents_num = 0

    for sent in example["document"]:
        sents += f" {sent}"
        sents_num += len(sent_tokenize(sent))
        if sents_num >= 5:
            summaries += summary_sents(sents)
            sents_num = 0
            sents = str()

    if sents_num > 0:
        summaries += summary_sents(sents)

    example["summary"] = summaries

    return example

In [7]:
def summary_example_all(context, document):
    if context is None:
        summaries = []
    elif context == "[]" or len(context) < 10:
        summaries = []
    else:
        summaries = [context]
    print(summaries)
    summaries += summary_sents(" ".join(document))

    return summaries

In [8]:
BATCH_SIZE = 10000

In [9]:
def summary_batch(batch):
    for i in range(BATCH_SIZE):
        batch["context"][i] = summary_example_all(
            batch["context"][i], batch["document"][i]
        )
    return batch

In [10]:
summary_dataset = dataset.map(summary_batch, batched=True, batch_size=BATCH_SIZE)

Map:   0%|          | 0/151879 [00:00<?, ? examples/s]

[]
[]
["[' Thời gian thi công nhà và hiệu lực của hợp đồng là 04 tháng  Trong quá trình thi công thì nhà thầu thể hiện năng lực, kinh nghiệm yếu (thi công trễ tiến độ, đổ bê tông sàn không đạt yêu cầu về mặt kỹ thuật và mỹ thuật {bị thấm nước và bị xệ (võng)]; Về thanh toán tiền tạm ứng thì tôi đã ứng nhiều hơn rất nhiều so với khối lượng thi công hoàn thành (chỉ sắp xong phần thô)  Nay thời hạn hợp đồng đã hết hiệu lực, nhà thầu có dấu hiệu bỏ trốn (không tiếp tục thi công hoàn thiện), tôi cũng đã làm văn bản thông báo nhiều lần để thực hiện hợp đồng nhưng vẫn không được trả lời và thiện chí từ nhà thầu']"]
[]
[]
[]
["[' Câu hỏi của E như sau : E mới ra trường nên ko rõ lắm về hợp đồng thời vụ, CTy có làm hợp đồng thời vụ 3 tháng với một số người  có phải mỗi tháng khi trả lương E phải trích 10% để đóng thuế tncn ko, và nếu Cty E ko trích 10% tiền lương của những người lao động thời vụ thì tiền lương cty e tra cho những người làm thời vụ có được tính vào chi phí hợp lý khi tính thuế t

KeyboardInterrupt: 